# Measuring compilation time

UNDER WORK

## Globally, with linux `time` command

In the Linux world, information about the overall execution time of an application can be obtained by simply preceding the application name with the `time` command.

In [1]:
%%file tmp.chrono.1.h

#include <vector>
#include <cstdlib>
#include <cassert>
#include <iostream>

Overwriting tmp.chrono.1.h


In [2]:
%%file tmp.chrono.2.h

std::vector<double> generate( int size )
 {
  std::vector<double> datas(size) ;
  for ( double & data : datas )
   { data = std::rand()/(RAND_MAX+1.) ; }
  return datas ;
 }

Overwriting tmp.chrono.2.h


In [3]:
%%file tmp.chrono.3.h

double analyse( std::vector<double> const & datas, int power )
 {
  double res = 0 ;
  for ( double data : datas )
   {
    double val = 1 ; 
    for ( int j=0 ; j<power ; ++j )
      val *= data ;
    res += val ;
   }
  return res ;
 }

Overwriting tmp.chrono.3.h


In [4]:
%%file tmp.chrono.cpp

#include "tmp.chrono.1.h"
#include "tmp.chrono.2.h"
#include "tmp.chrono.3.h"

int main( int argc, char * argv[] )
 {
  assert(argc==3) ;
  int size {atoi(argv[1])} ;
  int power {atoi(argv[2])} ;

  auto datas = generate(size) ;
  auto res = analyse(datas,power) ;
  std::cout<<res<<std::endl ; 
 }

Overwriting tmp.chrono.cpp


In [5]:
%%file tmp.chrono.sh

rm -f tmp.chrono.exe
g++ -std=c++17 -O3 tmp.chrono.cpp -o tmp.chrono.exe
time ./tmp.chrono.exe $*

Overwriting tmp.chrono.sh


In [6]:
!bash -l ./tmp.chrono.sh 1024 100000

0.525744

real	0m2.457s
user	0m2.457s
sys	0m0.000s


Details of the `time` display:
- `real` : the elapsed time seen in real life.
- `user` : the execution time spent in the user code.
- `sys`  : the execution time spent in system calls.

## Repeated time measurement

When monitoring the execution time, especially a small code, and especially when running on a non-reserved dedicated machine :
- **run your program many times** and compute the mean execution time,
- **ensure each single run is long enough** so that the processor pipelines get filled and you go well beyond the initial computing latency.

Also, be aware that if your data size is larger than the CPU cache, this may reduce I/O throughput, thus make your program I/ bound. In such a case, optimizing the computation instructions will not improve your global execution time.

Below, we run the program once, so to check the result. Then we run it 10 times, measuring the execution time with a GNU flavor of `time`, and redirect the results into a python script, which will finally compute the mean time. We ask a power of `100000`, to be sure the arithmetic intensity is high, and we limit the size of the array to `1024`, below the typical cache size.

In [7]:
%%file tmp.chrono.sh

rm -f tmp.chrono.exe
g++ -std=c++17 -O3 tmp.chrono.cpp -o tmp.chrono.exe
./tmp.chrono.exe $*

rm -f tmp.chrono.py
echo "t = 0" >> tmp.chrono.py
for i in 0 1 2 3 4 5 6 7 8 9
do \time -f "t += %U" -a -o ./tmp.chrono.py ./tmp.chrono.exe $* >> /dev/null
done
echo "print('(~ {:.3f} s)'.format(t/10.))" >> tmp.chrono.py
python3 tmp.chrono.py

Overwriting tmp.chrono.sh


In [8]:
!bash -l tmp.chrono.sh 1024 100000

0.525744
tmp.chrono.sh: line 9: time: command not found
tmp.chrono.sh: line 9: time: command not found
tmp.chrono.sh: line 9: time: command not found
tmp.chrono.sh: line 9: time: command not found
tmp.chrono.sh: line 9: time: command not found
tmp.chrono.sh: line 9: time: command not found
tmp.chrono.sh: line 9: time: command not found
tmp.chrono.sh: line 9: time: command not found
tmp.chrono.sh: line 9: time: command not found
tmp.chrono.sh: line 9: time: command not found
(~ 0.000 s)


## Internal time measurement with `std::chrono`

When trying to speed-up a big real world application, one needs to known the execution time of the subparts of the application, so to focus his efforts where it is worth. **It is highly advised to learn how to use a profiling tool such as [`perf`](https://perf.wiki.kernel.org/index.php/Main_Page)**.

Meanwhile, some internal information can be collected thanks to `std::chrono`. One of the interesting aspects of `std::chrono` is the work done on time units. Below, we measure the execution time of `generate` and displays it in milliseconds.

In [9]:
%%file tmp.chrono.2.h

#include <chrono>
        
std::vector<double> generate( int size )
 {
  using namespace std::chrono ;
  auto t1 {steady_clock::now()} ;   

  std::vector<double> datas(size) ;     
  for ( double & data : datas )
   { data = std::rand()/(RAND_MAX+1.) ; }

  auto t2 {steady_clock::now()} ;
  auto dt {duration_cast<microseconds>(t2-t1).count()} ;
  std::cout<<"(generate: "<<dt<<" us)"<<std::endl ;
     
  return datas ;
 }

Overwriting tmp.chrono.2.h


In [10]:
!rm -f tmp.chrono.exe && g++ -std=c++17 -I./ tmp.chrono.cpp -o tmp.chrono.exe

In [11]:
!./tmp.chrono.exe 1024 100000

(generate: 29 us)
0.525744


## Comparing two codes with [QuickBench](https://www.quick-bench.com/)

If you do not care about the absolute computation time, but want to compare two (or more) alternative implementations, you may like the online tool [QuickBench](https://www.quick-bench.com/), powered by [Google Benchmark](https://github.com/google/benchmark).

Basically, in the QuickBench main window, for each code to be compared, add a section like this...

In [ ]:
static void CodeVersion1(benchmark::State& state) {

  // code to be executed once
  // ...
    
  for (auto _ : state) {

    // code to be measured repeatedly
    // ...
    
    // output variable should be protected so that
    //  the compiler does not optimize them out
    benchmark::DoNotOptimize(res) ;

  }

}
BENCHMARK(CodeVersion1);

.. and push the button *Run Benchmark*.

# Questions ?

# Exercise

The code below defines some kind of "meta-function" `time()`, which takes as input another function `f`, and a set of arguments to be used for a call to `f`.
1. You are asked to complete the definition of `time`, with `chrono` features, so to compute, display and compare the execution time of `analyse1` and `analyse2`.
2. Try to write a Python script which will run the program 10 times, and compute the mean execution time of `generate`, `analyse1` and `analyse2`...
3. ...And/or try with [QuickBench](https://www.quick-bench.com/).

In [ ]:
%%file tmp.chrono.cpp

#include <valarray>
#include <cstdlib>
#include <cassert>
#include <iostream>
#include <string_view>
#include <chrono>

template< typename Fonction, typename... ArgTypes >
auto time( std::string_view title, Fonction f, ArgTypes... args )
 {
  // to be completed
  auto res {f(args...)} ;
  // to be completed
  std::cout<<"("<<title<<" time: ?? us)"<<std::endl ;
  return res ;
 }

std::valarray<double> generate( int size )
 {
  std::valarray<double> datas(size) ;
  for ( double & data : datas )
   { data = std::rand()/(RAND_MAX+1.) ; }
  return datas ;
 }

double analyse1( std::valarray<double> const & datas, int power )
 {
  double res = 0 ;
  for ( double data : datas )
   {
    double val = 1 ; 
    for ( int j=0 ; j<power ; ++j )
      val *= data ;
    res += val ;
   }
  return res ;
 }

double analyse2( std::valarray<double> datas, int power )
 {
  std::valarray<double> vals(1.,datas.size()) ;
  for ( int j=0 ; j<power ; ++j )
   { vals *= datas ; }
  double res = 0 ;
  for ( double val : vals )
   { res += val ; }
  return res ;
 }

int main( int argc, char * argv[] )
 {
  assert(argc==3) ;
  int size {atoi(argv[1])} ;
  int power {atoi(argv[2])} ;

  auto datas = time("gen",generate,size) ;
  auto res1 = time("ana1",analyse1,datas,power) ;
  auto res2 = time("ana2",analyse2,datas,power) ;
  std::cout << res1 << " " << res2 << std::endl ;
 }

In [ ]:
!rm -f tmp.chrono.exe && g++ -O3 -std=c++17 -I./ tmp.chrono.cpp -o tmp.chrono.exe

In [ ]:
!./tmp.chrono.exe 1024 100000

# Resources

- [Chrono](https://en.cppreference.com/w/cpp/chrono)
- [QuickBench](https://www.quick-bench.com/)
- [Google Benchmark](https://github.com/google/benchmark)
- [Perf](https://perf.wiki.kernel.org/).

© *CNRS 2022*
*Assembled and written in french by David Chamont, translated by Karim Hasnaoui in 2021, this work is made available according to the terms of the [Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*